In [ ]:
%%capture cap --no-stderr

#### HARD ENTERED VALUES ###

TMaxDivFactor = 10 # Factor to divide circular orbital period at rmin
Nsteps = 8050 # Hard entered after COMMENTING OUT lines 49-50

print('### Times relevant to integration\n')
print('\n')

### Find length of each timestep for adequate resolution ###
print('STEP 1: Find length of each timestep for adequate resolution')
print('\n')

def timescalePhys(R,vc): # <-- use R [kpc] and vc [km/s]
    R = R *u.kpc
    vc = vc *u.km/u.s
    T = 2 *np.pi *R/vc
    return T.to(u.Myr)
def timescaleNat(R,vc): # <-- use natural units
    R = R
    vc = vc
    T = 2 *np.pi *R/vc 
    return T
dtmaxPhys = timescalePhys(rmin*ro,vcirc(mwp,rmin,vo=vo,ro=ro))/TMaxDivFactor
dtmaxNat  = timescaleNat(rmin,vcirc(mwp,rmin))/TMaxDivFactor

NatToGyrConversion = dtmaxPhys/dtmaxNat # <-- Conversion factor
print('For adequate resolution, want timestep to be')
print('   dtmaxPhys <'+str(np.round(dtmaxPhys,2))+' in physical units, which is equal to ')
print('   dtmaxNat <'+str(np.round(dtmaxNat/10,4))+' in natural units')
print('\n')

print('Use these to define conversion factor:')
print('   NatToGyrConversion = dtmaxPhys/dtmaxNat = ',np.round(NatToGyrConversion,2),'per natural time unit')
print('\n')

TSlowNat = ((TSlowPhys/NatToGyrConversion).decompose()).value
print('Slowing period for the bar is set to:')
print('   TSlowPhys = ',TSlowPhys,' in physical units and')
print('   TSlowNat = ',np.round(TSlowNat,2),' in natural units')
print('\n')

ntstepApprox = ((TSlowPhys/dtmaxPhys).decompose()).value
print('Therefore the total number of timesteps should be:')
print('   Nsteps > TSlow/dtmax = ',np.round(ntstepApprox,1))
# sNsteps = input("Please choose an integer number of steps that is greater than this number: ")
# Nsteps = int(sNsteps)

tstepPhys = TSlowPhys/Nsteps
tstepNat = ((tstepPhys/NatToGyrConversion).decompose()).value
print('The number of steps is set to Nsteps =',Nsteps)
print('   where each step has length tstepPhys ~',np.round(tstepPhys.to(u.Myr),3),'in physical units')
print('   and tstepNat ~',np.round(tstepNat,5),' in natural time units')
print('\n')

## Find times for each slowing bar rotation
print('STEP 2: Identify timestamp for each rotation of the slowing bar')
print('\n')

tvector = np.arange(to,TSlowNat,tstepNat)
print('The array measuring time in natural units spaced by tstepNat is called tvector')
print('   and has length len(tvector) = ',len(tvector))
print('\n')

TbariNat = 2.*np.pi/omegaBi
TbarfNat = 2.*np.pi/omegaBf
TbariPhys = (TbariNat*NatToGyrConversion).to(u.Gyr)
TbarfPhys = (TbarfNat*NatToGyrConversion).to(u.Gyr)
print('Inital pattern speed is omegaBi = ',np.round(omegaBi,2),' [natural units]')
print('   As such a single bar period is: (using T = 2 pi / Omega_B)')
print('   TbariNat =',np.round(TbariNat,2),' natural units')
print('   TbariPhys =',np.round(TbariPhys,2),' physical units')
print('   Corotation at CRi =',np.round( (TbariPhys*vcirc(mwp,CRi,vo=vo,ro=ro)*u.km/u.s/(2.*np.pi)).to(u.kpc), 2),' physical units')

print('Final pattern speed is omegaBf = ',np.round(omegaBf,2),' [natural units]')
print('   As such a single bar period is:')
print('   TbarfNat =',np.round(TbarfNat,2),' natural units')
print('   TbarfPhys =',np.round(TbarfPhys,2),' physical units')
print('   Corotation at CRf =',np.round( (TbarfPhys*vcirc(mwp,CRf,vo=vo,ro=ro)*u.km/u.s/(2.*np.pi)).to(u.kpc), 2),' physical units')
print('\n')

def findomegat(omegai, omegaf, t):
    dO = omegaf-omegai
    dt = t[-1]-t[0]#ttot
    omegat= dO/dt *(t-t[0]) + omegai
    return omegat 
omegat = findomegat(omegaBi,omegaBf,tvector)
print('The bar pattern speed is defined as:')
print('   omegat = [(omegaBf - omegaBi) / TSlow] x (t-t0) + omegaBi')
print('   where t0 is the initial time')
print('The array with length',len(omegat),'giving the time dependent pattern speed is named omegat')
# Plot test (pl.plot) shows linear decrease over time as expected

def findphit(omegai, omegat, t):
    phit = omegat*t + omegai*t[0]
    return phit
phit = findphit(omegaBi,omegat,tvector)
print('The angle of the bar pattern over time:')
print('   phit = omegat x (t-t0) + omegaBi x to')
print('   where phit is implicitly in units of radians')
print('The array with length',len(phit),'giving the time dependent angle of the bar is named phit')
print('\n')

cosphit = np.cos(phit*u.rad)
maxpoints = argrelextrema(cosphit, np.greater)
endmwCR = lindbladR(mwp,omegat[maxpoints[0][-1]],m='corotation')*ro*u.kpc
print('In order to take snapshot when the bar is aligned with x=0')
print('   must find the timestamp when phi=0 for each rotation.')
print('Since phi(to)=0, cos(pho(to)=1)')
print('   the local maxima in function cos(phit) will closely approximate x=0 for snapshots.')
print('The array called maxpoints gives the indices of the ',len(maxpoints[0]),' maxima in cos(phit).')
print('   This implies that the bar makes',len(maxpoints[0]),'full rotations in',TSlowPhys)
print('\n')

print('Important Note: The last index indicating x=0 is not the last step in the simulation,')
print('   rather it is at step i = ',maxpoints[0][-1],',')
print('   which is time',np.round(tvector[maxpoints[0][-1]],2),'in natural units and')
print('   and ',np.round((tvector[maxpoints[0][-1]]*NatToGyrConversion).to(u.Gyr),2),'in physical units.')
print('   This implies the final radius of corotation is R_CR=',np.round(endmwCR,2))

print('\n')

print('For reference, these maxima correspond to phi/(2 pi) = ',np.round(phit[maxpoints]/(2.*np.pi),4))
pl.plot(cosphit)    
pl.scatter(maxpoints,np.ones(len(maxpoints[0])))

In [ ]:
with open(rmfile, 'a') as f:
   f.write(cap.stdout)

In [ ]:
# Setup meta data files
tfile = './orbits/'+str(nsample*nbatch)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(simname)+'_tvector.npy'
tphiofile = './orbits/'+str(nsample*nbatch)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(simname)+'_tphi0.npy'
omegatfile =  './orbits/'+str(nsample*nbatch)+'N_qiDF_'+dpType+'p_'+AAType+'_'+str(simname)+'_Omegat.npy'

np.save(tfile,tvector)
nphio = maxpoints[0]
nphio = np.insert(nphio,0,0)
np.save(tphiofile,nphio)
np.save(omegatfile,omegat[nphio])
